# Agenda

1. The iterator protocol
    - Adding iteration to your classes
    - Different techniques for that
2. Generator functions
3. Generator comprehensions
4. Decorators
5. Threading and multiprocessing 

In [1]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
        
p1 = Person('name1')        
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

Hello, name1!
Hello, name2!


In [2]:
import weakref

In [3]:
p3 = weakref.ref(p2)


In [4]:
p3

<weakref at 0x10c289310; to 'Person' at 0x10c2874c0>

In [5]:
p3.ref

AttributeError: 'weakref' object has no attribute 'ref'

In [7]:
w = weakref.ref(Person('name10'))

In [8]:
w

<weakref at 0x10b2aabd0; dead>

In [9]:
type(w)

weakref

In [10]:
dir(w)

['__call__',
 '__callback__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [11]:
p3 = weakref.ref(p2)


In [13]:
p3

<weakref at 0x10c289310; to 'Person' at 0x10c2874c0>

In [14]:
p1 = Person('name1')
p2 = Person('name2')

all_people = {1: p1, 2: p2}

In [15]:
del(p1)

In [16]:
len(all_people)

2

In [17]:
p1 = Person('name1')
p2 = Person('name2')

all_people = weakref.WeakValueDictionary()

all_people[1] = p1
all_people[2] = p2

In [18]:
all_people

In [20]:
list(all_people.items())

[(1, <__main__.Person at 0x10c263760>), (2, <__main__.Person at 0x10c2636a0>)]

In [21]:
len(all_people)

2

In [22]:
del(p1)

In [23]:
len(all_people)

2

In [24]:
len(all_people)

2

In [25]:
list(all_people.items())

[(1, <__main__.Person at 0x10c263760>), (2, <__main__.Person at 0x10c2636a0>)]

In [30]:
p1 = Person('name1')
p2 = Person('name2')

all_people = weakref.WeakValueDictionary()

all_people[1] = p1
all_people[2] = p2

In [31]:
del(p1)

In [32]:
len(all_people)

1

In [33]:
del(p2)

In [34]:
len(all_people)

0

# Iterator protocol

In [35]:
s = 'abcde'

for one_character in s:
    print(one_character)

a
b
c
d
e


# Protocol

1. Ask an object if it's iterable (`iter`)
    - Returns an iterator object if it is iterable
    - Raises an exception if it's not iterable
2. Ask the iterator we got back for its next item (`next`)
3. Each time we call `next`, we'll get an object
4. When we reach the end of the iteration, we get... the `StopIteration` exception

In [36]:
iter(s)

In [37]:
iter(s)

In [38]:
iter(s)

In [39]:
i = iter(s)  # store the string iterator in "i"

In [40]:
next(i)

'a'

In [41]:
next(i)

'b'

In [42]:
next(i)

'c'

In [43]:
next(i)

'd'

In [44]:
next(i)

'e'

In [45]:
next(i)

StopIteration: 

In [46]:
for i in 10:
    print(i)

TypeError: 'int' object is not iterable

In [47]:
iter(10)

TypeError: 'int' object is not iterable

In [55]:
class MyIterator:
    def __init__(self, data):
        print(f'Now in MyIterator.__init__ with {data=}')
        self.data = data
        self.index = 0
        
    def __iter__(self):
        print(f'Now in MyIterator.__iter__')
        return self   # the object is its own iterator!
    
    def __next__(self):
        print(f'Now entering MyIterator.__next__')
        if self.index >= len(self.data):
            print(f'\t{self.index=}, ending the loop')
            raise StopIteration
            
        value = self.data[self.index]
        print(f'\t{self.index=}, {value=}')
        self.index += 1
        return value

m = MyIterator('abc')

for one_item in m:
    print(one_item)

Now in MyIterator.__init__ with data='abc'
Now in MyIterator.__iter__
Now entering MyIterator.__next__
	self.index=0, value='a'
a
Now entering MyIterator.__next__
	self.index=1, value='b'
b
Now entering MyIterator.__next__
	self.index=2, value='c'
c
Now entering MyIterator.__next__
	self.index=3, ending the loop


In [51]:
f = open('/etc/passwd')

iter(f) is f

True

# Exercise: Circle

1. Create a class, `Circle`, which takes two arguments:
    - an iterable piece of data (`data`)
    - an integer (`maxtimes`)
2. If we run a `for` loop on an instance of `Circle`, we'll get `maxtimes` results back.
3. The return values will come from `data`.
    - If the number of elements in `data` is larger than `maxtimes`, then we'll just end after `maxtimes`, producing one value at a time.
    - If the number of elements in `data` is smaller than `maxtimes`, then when we get to the end of the data, we'll come back to index 0 -- thus going around and around until we give the number of results indicated in `maxtimes`.
    
```python
c = Circle('abc', 7)

for one_item in c:
    print(one_item)
```

```
a
b
c
a
b
c
a
```

In [57]:
class Circle:
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        self.index = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index >= self.maxtimes:
            raise StopIteration
            
        value = self.data[self.index % len(self.data)]
        self.index += 1
        return value
    
c = Circle('abc', 7)

for one_item in c:
    print(one_item)

a
b
c
a
b
c
a


In [58]:
0 % 3  # remainder from division of 0/3

0

In [59]:
1 % 3  # remainder from 1/3

1

In [60]:
2 % 3

2

In [61]:
3 % 3

0

In [62]:
4 % 3

1

In [63]:
5 % 3

2

In [64]:
mylist = [10, 20, 30, 40, 50]

i1 = iter(mylist)
i2 = iter(mylist)

In [65]:
i1

In [66]:
i2

In [67]:
next(i1)

10

In [68]:
next(i2)

10

In [69]:
next(i2)

20

In [71]:
next(i2)

30

In [72]:
next(i1)

20

In [74]:
c = Circle('abc', 7)

i1 = iter(c)
i2 = iter(c)

In [75]:
next(i1)

'a'

In [76]:
next(i1)

'b'

In [77]:
next(i2)

'c'

In [78]:
next(i2)

'a'

In [79]:
next(i1)

'b'

In [80]:
i1 is i2

True

In [81]:
i1 is c

True

In [83]:
c = Circle('abc', 7)

print('*** A ****')
for one_item in c:
    print(one_item, end= ' ')
    
print()

print('*** B ****')
for one_item in c:
    print(one_item, end= ' ')


*** A ****
a b c a b c a 
*** B ****


In [88]:
class CircleIterator:
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        self.index = 0

    def __next__(self):
        if self.index >= self.maxtimes:
            raise StopIteration
            
        value = self.data[self.index % len(self.data)]
        self.index += 1
        return value
    
class Circle:
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        
    def __iter__(self):
        return CircleIterator(self.data, self.maxtimes)
    
    
c = Circle('abc', 7)

print("*** A ****")
for one_item in c:
    print(one_item)
    
print("*** B ****")
for one_item in c:
    print(one_item)
    


*** A ****
a
b
c
a
b
c
a
*** B ****
a
b
c
a
b
c
a


In [85]:
iter(c)

In [86]:
iter(c)

In [87]:
iter(c)

In [89]:
class CircleIterator:
    def __init__(self, circle):
        self.circle = circle
        self.index = 0

    def __next__(self):
        if self.index >= self.circle.maxtimes:
            raise StopIteration
            
        value = self.circle.data[self.index % len(self.circle.data)]
        self.index += 1
        return value
    
class Circle:
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        
    def __iter__(self):
        return CircleIterator(self)
    
    
c = Circle('abc', 7)

print("*** A ****")
for one_item in c:
    print(one_item)
    
print("*** B ****")
for one_item in c:
    print(one_item)
    


*** A ****
a
b
c
a
b
c
a
*** B ****
a
b
c
a
b
c
a


In [91]:
# what if we set self.index = 0 just before raising the exception?

class Circle:
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        self.index = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index >= self.maxtimes:
            self.index = 0
            raise StopIteration
            
        value = self.data[self.index % len(self.data)]
        self.index += 1
        return value
    
c = Circle('abc', 7)

print(f'*** A ***')
for one_item in c:
    print(one_item)

print(f'*** B ***')  
for one_item in c:
    print(one_item)

*** A ***
a
b
c
a
b
c
a
*** B ***
a
b
c
a
b
c
a


In [92]:
c = Circle('abc', 7)

i1 = iter(c)
i2 = iter(c)

In [93]:
i1 is i2

True

In [94]:
next(i1)

'a'

In [95]:
next(i1)

'b'

In [96]:
next(i2)

'c'

In [97]:
next(i2)

'a'

# Exercise: MyRange

1. Create a class, `MyRange`, which takes 1, 2, or 3 arguments.  It'll work much like `range` does.
2. If we call it with 1 argument, then we expect the iterations to run from 0 up to (and not including) that number.
3. If we call it with 2 arguments, then we expect the iterations to run from the first number up to (and not including) the second.
4. If we call it with 3 arguments, then we expect the iterations to run from the first to the second, with a step size of the third.
5. Use the two-class method for iterator construction in creating this.

In [98]:
list(range(5))

[0, 1, 2, 3, 4]

In [99]:
list(range(5, 10))

[5, 6, 7, 8, 9]

In [100]:
list(range(5, 20, 2))

[5, 7, 9, 11, 13, 15, 17, 19]

In [106]:
class MyRangeIterator:
    def __init__(self, current, stop, step):
        self.current = current
        self.stop = stop
        self.step = step
    
    def __next__(self):
        if self.current >= self.stop:
            raise StopIteration
            
        value = self.current
        self.current += self.step
        
        return value
    

class MyRange:
    def __init__(self, first, second=None, step=1):
        if second is None:
            self.current = 0
            self.stop = first
        else:
            self.current = first
            self.stop = second
            
        self.step = step
        
    def __iter__(self):
        return MyRangeIterator(self.current, self.stop, self.step)
    
list(MyRange(5))

[0, 1, 2, 3, 4]

In [107]:
list(MyRange(5, 10))

[5, 6, 7, 8, 9]

In [108]:
list(MyRange(5, 20, 2))

[5, 7, 9, 11, 13, 15, 17, 19]

In [102]:
list('abc')

['a', 'b', 'c']

In [103]:
list((10, 20, 309))

[10, 20, 309]

 # Generator functions

In [109]:
def myfunc():
    return 1
    return 2
    return 3

In [110]:
myfunc()

1

In [111]:
import dis
dis.dis(myfunc)

  2           0 LOAD_CONST               1 (1)
              2 RETURN_VALUE


In [112]:
def myfunc():
    yield 1
    yield 2
    yield 3

In [113]:
myfunc()

<generator object myfunc at 0x10ce27f90>

In [114]:
g = myfunc()

In [115]:
type(g)

generator

In [116]:
next(g)

1

In [117]:
next(g)

2

In [118]:
next(g)

3

In [119]:
next(g)

StopIteration: 

In [120]:
def myfunc():
    yield 1
    yield 2
    yield 3

Running `next` on a generator object executes the function body through the next `yield` statement. 

The returned value from `next` is whatever `yield` returned.

If we hit the end of the function body, the generator raises `StopIteration`.

In [122]:
def fib():   # "generator function," but many people just call it a "generator"
    first = 0
    second = 1
    while True:
        yield first
        first,second = second, first+second

In [123]:
g = fib()

In [124]:
next(g)

0

In [125]:
next(g)

1

In [126]:
next(g)

1

In [127]:
next(g)

2

In [132]:
g = fib()
for i in range(100):
    print(next(g), end=' ')

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 12586269025 20365011074 32951280099 53316291173 86267571272 139583862445 225851433717 365435296162 591286729879 956722026041 1548008755920 2504730781961 4052739537881 6557470319842 10610209857723 17167680177565 27777890035288 44945570212853 72723460248141 117669030460994 190392490709135 308061521170129 498454011879264 806515533049393 1304969544928657 2111485077978050 3416454622906707 5527939700884757 8944394323791464 14472334024676221 23416728348467685 37889062373143906 61305790721611591 99194853094755497 160500643816367088 259695496911122585 420196140727489673 679891637638612258 1100087778366101931 1779979416004714189 2880067194370816120 4660046610375530309 754011380474634642

In [135]:
for one_item in fib():
    print(one_item, end=' ')
    if one_item > 100_000_000_000_000_000:
        break

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 12586269025 20365011074 32951280099 53316291173 86267571272 139583862445 225851433717 365435296162 591286729879 956722026041 1548008755920 2504730781961 4052739537881 6557470319842 10610209857723 17167680177565 27777890035288 44945570212853 72723460248141 117669030460994 190392490709135 308061521170129 498454011879264 806515533049393 1304969544928657 2111485077978050 3416454622906707 5527939700884757 8944394323791464 14472334024676221 23416728348467685 37889062373143906 61305790721611591 99194853094755497 160500643816367088 

# Exercise: get_vowels

1. Write a generator function, `get_vowels`, that takes a filename as an argument.
2. With each iteration, return the next vowel (a, e, i, o, or u) from the file that we're reading from.
3. When we reach the end of the file and/or there are no more vowels, then the generator should exit.

Never raise `StopIteration` in a generator function! It happens automatically when the function exits.  You can use `return` by itself to do that prematurely.

In [140]:
def get_vowels(filename):
    for one_line in open(filename):
        for one_character in one_line.lower():
            if one_character in 'aeiou':
                yield one_character

In [142]:
g = get_vowels('/etc/passwd')

In [143]:
for one_vowel in g:
    print(one_vowel)

u
e
a
a
a
e
o
e
a
i
i
e
i
o
u
e
i
e
o
e
e
e
i
u
i
i
i
e
u
e
o
e
a
o
e
i
e
i
i
o
a
i
o
i
o
i
e
o
e
i
e
o
e
e
e
o
e
i
e
o
a
a
e
o
a
i
i
o
a
i
o
a
i
o
a
o
u
o
e
i
e
o
o
o
u
i
i
e
e
u
e
a
e
u
i
a
e
o
o
e
a
i
i
a
o
a
o
o
i
a
e
o
e
e
i
e
a
o
o
u
i
a
e
u
u
u
i
o
u
i
o
o
o
o
a
o
o
u
u
u
i
u
u
i
o
a
a
e
a
a
e
a
e
o
a
e
u
i
a
e
e
o
e
o
e
i
e
a
e
o
u
i
a
e
i
a
a
i
a
i
a
a
i
a
a
e
u
i
a
e
i
i
e
i
e
a
o
o
u
u
i
a
e
o
i
o
i
a
i
e
e
a
o
o
o
i
u
i
a
e
e
i
e
o
i
u
a
i
o
e
i
e
a
e
u
i
a
e
e
e
i
i
a
e
e
o
e
e
i
e
a
e
u
i
a
e
a
o
e
a
a
o
e
e
i
e
a
a
o
e
u
i
a
e
a
a
a
u
a
e
u
i
a
e
a
e
e
e
a
e
e
e
a
e
o
a
e
u
i
a
e
e
o
e
o
e
i
e
a
e
o
a
e
o
u
i
a
e
e
o
e
e
o
e
o
u
e
a
i
o
a
e
u
i
a
e
a
o
e
a
e
a
e
u
i
a
e
e
o
e
e
o
e
a
e
u
i
a
e
a
a
e
e
o
e
e
o
a
e
u
i
a
e
o
i
e
e
e
e
i
a
e
e
e
u
i
a
e
e
a
e
e
e
u
e
a
e
u
i
a
e
e
e
a
e
u
i
a
e
e
e
a
e
u
i
a
e
e
e
a
e
u
i
a
e
i
i
e
e
e
a
a
i
o
a
e
u
i
a
e
u
i
i
e
e
a
i
e
e
a
e
u
i
a
e
u
u
a
i
i
a
o
a
i
a
u
i
a
e
a
i
a
a
i
a
i
e
e
a
e
u
i
a
e
a
e
e
a
i
a
i
o
e
e
a
e
u
i
a
e


In [144]:
return 5

SyntaxError: 'return' outside function (<ipython-input-144-be15c3e79f52>, line 1)

In [145]:
break

SyntaxError: 'break' outside loop (<ipython-input-145-6aaf1f276005>, line 1)

In [146]:
continue

SyntaxError: 'continue' not properly in loop (<ipython-input-146-6ca52a340915>, line 1)

In [147]:
dis.show_code(get_vowels)

Name:              get_vowels
Filename:          <ipython-input-140-0d2286702179>
Argument count:    1
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  3
Stack size:        4
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE
Constants:
   0: None
   1: 'aeiou'
Names:
   0: open
   1: lower
Variable names:
   0: filename
   1: one_line
   2: one_character


In [148]:
g = get_vowels('/etc/passwd')

In [149]:
next(g)

'u'

In [150]:
next(g)

'e'

In [151]:
next(g)

'a'

In [152]:
g

<generator object get_vowels at 0x10ce3c430>

In [153]:
dir(g)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [154]:
g.gi_running

False

In [155]:
g.gi_code

<code object get_vowels at 0x10c269500, file "<ipython-input-140-0d2286702179>", line 1>

In [156]:
g.gi_code.co_varnames

('filename', 'one_line', 'one_character')

In [157]:
g.gi_code.co_argcount

1

In [158]:
g.gi_code.co_code

b't\x00|\x00\x83\x01D\x00] }\x01|\x01\xa0\x01\xa1\x00D\x00]\x12}\x02|\x02d\x01v\x00r\x14|\x02V\x00\x01\x00q\x14q\x08d\x00S\x00'

In [159]:
dis.dis(g.gi_code.co_code)

          0 LOAD_GLOBAL              0 (0)
          2 LOAD_FAST                0 (0)
          4 CALL_FUNCTION            1
          6 GET_ITER
    >>    8 FOR_ITER                32 (to 42)
         10 STORE_FAST               1 (1)
         12 LOAD_FAST                1 (1)
         14 LOAD_METHOD              1 (1)
         16 CALL_METHOD              0
         18 GET_ITER
    >>   20 FOR_ITER                18 (to 40)
         22 STORE_FAST               2 (2)
         24 LOAD_FAST                2 (2)
         26 LOAD_CONST               1 (1)
         28 CONTAINS_OP              0
         30 POP_JUMP_IF_FALSE       20
         32 LOAD_FAST                2 (2)
         34 YIELD_VALUE
         36 POP_TOP
         38 JUMP_ABSOLUTE           20
    >>   40 JUMP_ABSOLUTE            8
    >>   42 LOAD_CONST               0 (0)
         44 RETURN_VALUE


In [160]:
get_vowels??

In [161]:
type(g.gi_frame)

frame

In [162]:
dir(g.gi_frame)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'f_back',
 'f_builtins',
 'f_code',
 'f_globals',
 'f_lasti',
 'f_lineno',
 'f_locals',
 'f_trace',
 'f_trace_lines',
 'f_trace_opcodes']

In [163]:
g.gi_frame.f_locals

{'filename': '/etc/passwd',
 'one_line': '# User Database\n',
 'one_character': 'a'}

In [164]:
next(g)

'a'

In [165]:
g.gi_frame.f_locals

{'filename': '/etc/passwd',
 'one_line': '# User Database\n',
 'one_character': 'a'}

In [166]:
next(g)

'a'

In [167]:
next(g)

'e'

In [168]:
g.gi_frame.f_locals

{'filename': '/etc/passwd',
 'one_line': '# User Database\n',
 'one_character': 'e'}

In [169]:
g.gi_frame.f_lineno

5

In [170]:
get_vowels??

# Exercise: read_n

1. Define `read_n` to be a generator function that takes two arguments:
    - `filename`, a string
    - `n`, an integer
2. Normally, when you iterate over a file, you get each line, one at a time